In [1]:
import yfinance as yf
import pandas_ta
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

ticker = "AAPL"

start_date = "2022-12-01"

end_date = "2024-01-02"

# Get the historical data for the ticker

stock_data = yf.download(ticker, start=start_date, end=end_date)

# Calculate technical indicators using pandas-ta
stock_data.ta.macd(append=True)
stock_data.ta.midpoint(append=True)
stock_data.ta.midprice(append=True)
stock_data.ta.vwap(append=True)
stock_data.ta.vwma(append=True)
stock_data.ta.vwma(append=True,length=20)
stock_data.ta.mfi(append=True)
stock_data['DELTA_10'] = stock_data['Close'] - stock_data['VWMA_10']

stock_data.reset_index(drop=True,inplace=True)

sd_scaler = StandardScaler()
# ct = ColumnTransformer([
#         ('somename', StandardScaler(), ['DELTA_10'])
#     ], remainder='passthrough')
# 
# feature = stock_data['DELTA_10'].to_frame()
# 
# sd_delta = ct.fit_transform(feature)
# print(sd_delta)
sd_delta = sd_scaler.fit_transform(stock_data)

print(type(sd_delta))
# print(feature)
# print(type(feature))

# stock_data.ta.rsi(append=True)
# stock_data.ta.bbands(append=True)
# stock_data.ta.obv(append=True)
# # 
# # # Calculate additional technical indicators
# stock_data.ta.sma(length=20, append=True)
# stock_data.ta.ema(length=50, append=True)
# stock_data.ta.stoch(append=True)
# stock_data.ta.adx(append=True)
# 
# stock_data.ta.ad(append=True)
# stock_data.ta.stdev(append=True)
# stock_data.ta.vwap(append=True)
# help(stock_data.ta)
# stock_data.index
# print(stock_data)
# stock_data.loc['2022-04-19']

# print(stock_data)

[*********************100%%**********************]  1 of 1 completed
<class 'numpy.ndarray'>


In [2]:
import sqlalchemy
import yfinance as yf
import pandas as pd
import pandas_ta
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

ticker='PLTR'
stock = yf.Ticker(ticker)
stock = stock.history(period="max")

rsi = yf.TA.RSI(stock)
print(rsi)


# Set the ticker symbol and the start and end dates

# ticker = "AAPL"
# 
# start_date = "2022-01-01"
# 
# end_date = "2022-04-25"
# 
# # Get the historical data for the ticker
# 
# ticker_data = yf.download(ticker, start=start_date, end=end_date)
# 
# # Calculate the RSI for the ticker data
# 
# rsi = yf.TA.RSI(ticker_data)
# 
# # Print the RSI
# 
# print(rsi)

rolling_averages = stock.rolling(5).mean()


stock["RollingClose_5"] = rolling_averages["Close"]

stock["RollingVolume_5"] = rolling_averages["Volume"]


rolling_averages = stock.rolling(120).mean()
stock["RollingClose_120"] = rolling_averages["Close"]
stock["RollingVolume_120"] = rolling_averages["Volume"]
stock["ticker"]=ticker
stock['PV_DERIVATIVE'] = 0.0
# print(len(stock))
# print(stock.shape)
# stock.at[0,'PV_DERIVATIVE'] = 0
# 
preClose = 0
preVolume =0
count = 0
for i, row in stock.iterrows():
    if count > 0 :
        absDerivative = (row['Close'] - preClose)/(row['Volume'] - preVolume)
        # normDerivative = absDerivative /stock.iloc[i].loc['RollingClose_120'] * stock.loc[i,'RollingVolume_120']
        normDerivative = absDerivative * 1000000000
        stock.at[i,'PV_DERIVATIVE'] = round(normDerivative,6)
    preVolume = row['Volume']
    preClose = row['Close']
    count += 1

    # print(round(normDerivative,6))


# print(stock.loc[0,:])
# print(stock.loc[1,:])
# print(stock.loc[808,:])
# print(stock.loc[[808]])
# print(type(stock))

engine = sqlalchemy.create_engine('sqlite:///D:\\sqlite\\data\\finance.db')

stock.to_sql('STOCK_L1',engine,if_exists='append')
# stock.to_sql('STOCK_L1',engine)
# print(stock['PV_DERIVATIVE'])



  


AttributeError: module 'yfinance' has no attribute 'TA'

In [1]:
import pandas as pd
data = pd.DataFrame({'Name' : [3, 4,6], 'Age' : [18, 92,98], 'Weight' : [68, 59,49]})

col_names = ['Name', 'Age', 'Weight']
features = data[col_names]

print(features)
print(type(features))

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

ct = ColumnTransformer([
        ('somename', StandardScaler(), ['Age', 'Weight'])
    ], remainder='passthrough')

ct.fit_transform(features)

   Name  Age  Weight
0     3   18      68
1     4   92      59
2     6   98      49
<class 'pandas.core.frame.DataFrame'>


array([[-1.41100443,  1.20270298,  3.        ],
       [ 0.62304092,  0.04295368,  4.        ],
       [ 0.78796352, -1.24565666,  6.        ]])